In [5]:
import tkinter as tk
from tkinter import ttk, filedialog, messagebox
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.preprocessing import MinMaxScaler

# Function to read the dataset and preprocess it
def load_dataset(file_path):
    try:
        # Read the dataset
        data = pd.read_csv(file_path)

        # Ensure required columns are present
        if 'duration' not in data.columns or 'rating' not in data.columns or 'title' not in data.columns:
            raise ValueError("Dataset must contain 'duration', 'rating', and 'title' columns.")

        # Extract numeric values from duration (e.g., "90 min" → 90, "2 Season" → 2)
        data['duration'] = data['duration'].str.extract(r'(\d+)', expand=False).astype(float)

        # Normalize or categorize ratings (assuming higher ratings are better)
        data['rating'] = pd.to_numeric(data['rating'], errors='coerce').fillna(0)

        # Remove rows with missing or invalid duration/rating
        data = data.dropna(subset=['duration', 'rating'])

        return data
    except Exception as e:
        messagebox.showerror("Error", f"Failed to load dataset: {e}")
        return None

# Function to train the SVM model
def train_model(data):
    try:
        # Define the target variable: Classify based on duration and rating
        data['target'] = ((data['duration'] < 90) & (data['rating'] > 7)).astype(int)

        # Features and target
        X = data[['duration', 'rating']]
        y = data['target']

        # Normalize the features
        scaler = MinMaxScaler()
        X = scaler.fit_transform(X)

        # Split the dataset
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

        # Train the SVM model
        model = SVC(probability=True)
        model.fit(X_train, y_train)

        return model, scaler
    except Exception as e:
        messagebox.showerror("Error", f"Model training failed: {e}")
        return None, None

# Function to recommend movies
def recommend_movies():
    try:
        # Predict the target class for all movies
        X = scaler.transform(movie_data[['duration', 'rating']])
        movie_data['score'] = svm_model.predict_proba(X)[:, 1]

        # Filter and sort movies by score, then take top 5
        recommended_movies = (
            movie_data[movie_data['score'] > 0.5]
            .sort_values(by=['score', 'rating'], ascending=[False, False])
            .head(5)
        )

        if recommended_movies.empty:
            result_text.delete(1.0, tk.END)
            result_text.insert(tk.END, "No movies found for the selected criteria.")
        else:
            sorted_movies = recommended_movies['title'].tolist()
            result_text.delete(1.0, tk.END)
            result_text.insert(tk.END, "\n".join(sorted_movies))
    except Exception as e:
        messagebox.showerror("Error", f"Failed to recommend movies: {e}")

# Function to select dataset file and train the model
def select_file():
    file_path = filedialog.askopenfilename(filetypes=[("CSV files", "*.csv")])
    if file_path:
        global movie_data, svm_model, scaler
        movie_data = load_dataset(file_path)
        if movie_data is not None:
            svm_model, scaler = train_model(movie_data)
            if svm_model and scaler:
                messagebox.showinfo("Success", "Dataset loaded and model trained successfully!")

# Create the GUI window
root = tk.Tk()
root.title("Movie Recommendation System")
root.geometry("600x400")

# Variables
movie_data = None
svm_model = None
scaler = None

# GUI Layout
frame = ttk.Frame(root, padding="10")
frame.grid(row=0, column=0, sticky="NSEW")

# File selection button
file_button = ttk.Button(frame, text="Select Dataset", command=select_file)
file_button.grid(row=0, column=0, pady=10, sticky="W")

# Recommend button
recommend_button = ttk.Button(frame, text="Recommend Top 5 Movies", command=recommend_movies)
recommend_button.grid(row=1, column=0, columnspan=2, pady=10)

# Scrollable results display
result_frame = ttk.Frame(frame)
result_frame.grid(row=2, column=0, columnspan=2, pady=10, sticky="NSEW")

result_text = tk.Text(result_frame, wrap=tk.WORD, height=15, width=50)
result_text.grid(row=0, column=0, sticky="NSEW")

scrollbar = ttk.Scrollbar(result_frame, orient=tk.VERTICAL, command=result_text.yview)
scrollbar.grid(row=0, column=1, sticky="NS")

result_text.config(yscrollcommand=scrollbar.set)

# Run the GUI main loop
root.mainloop()
